# 🛠️ 따릉이 매너 온도: 고장률로 살펴보는 서울의 따릉이 관리 성적표

In [ ]:
# 개발환경 셋팅
!pip install koreanize-matplotlib missingno scikit-learn

import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import matplotlib.patheffects as path_effects
import matplotlib.colors as mcolors
import koreanize_matplotlib
import seaborn as sns

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 45.8 MB/s eta 0:00:00


## 1. 데이터 준비

In [ ]:
# 데이터 불러오기
fault_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/프로젝트/final.csv', encoding='cp949')
rental_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/프로젝트/total_by_county.csv')

# 데이터 백업
fault_df_copy = fault_df.copy()
rental_df_copy = rental_df.copy()

In [ ]:
# 하위 3곳 시간대별 고장접수 현황 (5번 차트, 라인 차트)

# 🔹 1. 데이터 전처리 및 그룹화
rental_grouped = rental_df.groupby("자치구")["자전거번호"].count().reset_index()
rental_grouped.columns = ["자치구", "대여건수"]

fault_grouped = fault_df.groupby("자치구")["자전거번호"].count().reset_index()
fault_grouped.columns = ["자치구", "고장건수"]

# 🔹 2. 데이터 병합
merged_df = pd.merge(rental_grouped, fault_grouped, on="자치구", how="left").fillna(0)

# 🔹 3. 매너온도 계산 (100이 아닌 10000을 곱한 방식 적용)
merged_df["고장률(%)"] = (merged_df["고장건수"] / merged_df["대여건수"]) * 100
merged_df["매너온도"] = 10000 - (merged_df["고장률(%)"] * 100)  # 정확한 차이를 반영

# 🔹 4. 매너온도가 가장 낮은 3곳 선정
merged_df = merged_df.sort_values(by="매너온도", ascending=True).reset_index(drop=True)
lowest_3_districts = merged_df.head(3)["자치구"].tolist()  # 하위 3곳 추출

# 🔹 5. 시간대별 고장 건수 데이터 생성
fault_df["시간대"] = pd.to_datetime(fault_df["등록일시"]).dt.hour  # 시간대 추출
time_fault_df = fault_df.groupby(["시간대", "자치구"])["자전거번호"].count().reset_index()
time_fault_df.columns = ["시간대", "자치구", "고장건수"]

# 🔹 6. 하위 3곳의 시간대별 고장 건수 추출
time_pivot_df = time_fault_df.pivot_table(index="시간대", columns="자치구", values="고장건수", aggfunc="sum").fillna(0)

# 🔹 7. 전체 평균 컬럼 추가
time_pivot_df["전체 평균"] = time_pivot_df.mean(axis=1)

# 🔹 8. 하위 3곳의 컬럼만 선택하여 최종 테이블 생성
final_time_fault_df = time_pivot_df[["전체 평균"] + lowest_3_districts]

final_time_fault_df.head(24)

자치구,전체 평균,서초구,강남구,금천구
시간대,,,,
0,9.16,4.0,8.0,6.0
1,6.24,7.0,5.0,2.0
2,4.08,6.0,7.0,3.0
3,2.32,1.0,1.0,1.0
4,2.84,1.0,3.0,0.0
5,5.84,5.0,9.0,3.0
6,11.56,8.0,15.0,3.0
7,26.32,20.0,14.0,21.0
8,43.96,31.0,40.0,41.0


In [ ]:
# 원하는 열 순서 지정
column_order = ["전체 평균", "금천구", "강남구", "서초구"]

# 데이터프레임 열 순서 변경
final_time_fault_df = final_time_fault_df[column_order]

final_time_fault_df.head(24)

자치구,전체 평균,금천구,강남구,서초구
시간대,,,,
0,9.16,6.0,8.0,4.0
1,6.24,2.0,5.0,7.0
2,4.08,3.0,7.0,6.0
3,2.32,1.0,1.0,1.0
4,2.84,0.0,3.0,1.0
5,5.84,3.0,9.0,5.0
6,11.56,3.0,15.0,8.0
7,26.32,21.0,14.0,20.0
8,43.96,41.0,40.0,31.0


In [ ]:
#  CSV 파일 저장
output_path = "/content/drive/MyDrive/Colab Notebooks/프로젝트/final_time_fault.csv"
final_time_fault_df.to_csv(output_path, index=True, encoding="utf-8-sig")

In [ ]:
# 하위 3곳 시간대별 고장접수 현황 (6번 차트, 막대 차트)

# 기존 코드에서 이미 추출한 매너온도 하위 3곳 리스트 활용
lowest_3_districts = ["금천구", "강남구", "서초구"]  # 매너온도 가장 낮은 3개 자치구 (기존 코드 기반)

# 시간대 변환 (등록일시에서 시간만 추출)
fault_df["시간대"] = pd.to_datetime(fault_df["등록일시"]).dt.hour

# 주요 시간대 정의
def categorize_time(hour):
    if 6 <= hour < 9:
        return "출근 시간"
    elif 9 <= hour < 15:
        return "낮 시간"
    elif 15 <= hour < 18:
        return "오후 시간"
    elif 18 <= hour < 21:
        return "퇴근 시간"
    elif 21 <= hour < 24:
        return "야간 시간"
    else:
        return "심야 시간"

fault_df["주요시간대"] = fault_df["시간대"].apply(categorize_time)

# 주요 시간대별 고장건수 집계
time_summary_df = fault_df.groupby(["주요시간대", "자치구"])["자전거번호"].count().reset_index()
time_summary_df.columns = ["주요시간대", "자치구", "고장건수"]

# 피벗 테이블 생성 (행: 주요시간대, 열: 자치구, 값: 고장건수)
time_pivot_df = time_summary_df.pivot_table(index="주요시간대", columns="자치구", values="고장건수", aggfunc="sum").fillna(0)

# 전체 평균 추가
time_pivot_df["전체 평균"] = time_pivot_df.mean(axis=1)

# 하위 3곳의 컬럼만 선택하여 최종 테이블 생성
final_time_summary_df = time_pivot_df[["전체 평균"] + lowest_3_districts]

final_time_summary_df.head()

자치구,전체 평균,금천구,강남구,서초구
주요시간대,,,,
낮 시간,107.60,44,97,110
심야 시간,30.48,15,33,24
야간 시간,80.80,34,67,102
오후 시간,75.12,47,80,74
출근 시간,81.84,65,69,59


In [ ]:
# 원하는 행 순서 지정
row_order = ["출근 시간", "낮 시간", "오후 시간", "퇴근 시간", "야간 시간", "심야 시간"]

# 데이터프레임의 행 순서 변경
final_time_summary_df = final_time_summary_df.reindex(row_order)

# 변경된 데이터 확인 (최대 24개 행 표시)
final_time_summary_df.head(24)

자치구,전체 평균,금천구,강남구,서초구
주요시간대,,,,
출근 시간,81.84,65,69,59
낮 시간,107.60,44,97,110
오후 시간,75.12,47,80,74
퇴근 시간,142.20,94,160,141
야간 시간,80.80,34,67,102
심야 시간,30.48,15,33,24


In [ ]:
#  CSV 파일 저장
output_path = "/content/drive/MyDrive/Colab Notebooks/프로젝트/final_time_fault_summary.csv"
final_time_summary_df.to_csv(output_path, index=True, encoding="utf-8-sig")